# Introduction

1. In the Previous project object, DS was able to easily associate domains, as we did not have association
requests.This would mean that the DS needs to submit assocation requests to domains to connect them with each other.

2. Project Invitation is a useful feature, but is very hard to implement, when in a distrubted setting, let us assume the DS is able to create projects on domains without any request/approval on the Project Itself.

3. The Data Scientist is able to create a project if and only if , all the domains could talk to each other. (i.e they have beeen previously associated)

4. This would mean that in our semi-Decentralized, leader based system , when ever a node would like to add an event/message to the project, it has to be sent to the leader, which is then broadcasted to all the other nodes.

5. For other situations, for example when a node would like to asset metadata of another node in a Multi Domain User Code request, they could directly contact the node, to retrieve the info, instead of going through the leader.

6. This would require us to create a Full Mesh Network Topology, where each node is connected to each other.

In [ ]:
# third party
import pytest
from recordlinkage.datasets import load_febrl4

# syft absolute
import syft as sy
from syft.abstract_node import NodeType
from syft.service.network.routes import HTTPNodeRoute
from syft.service.response import SyftAttributeError
from syft.service.response import SyftSuccess

CANADA_DOMAIN_PORT = 9081
ITALY_DOMAIN_PORT = 9082
CANADA_ENCLAVE_PORT = 9083

# Launch nodes

We will begin by launching two domain nodes and an enclave node.

In [ ]:
canada_node = sy.orchestra.launch(
    name="canada-domain", port=CANADA_DOMAIN_PORT, dev_mode=True, reset=True
)
italy_node = sy.orchestra.launch(
    name="italy-domain", port=ITALY_DOMAIN_PORT, dev_mode=True, reset=True
)
canada_enclave = sy.orchestra.launch(
    name="canada-enclave",
    node_type=NodeType.ENCLAVE,
    port=CANADA_ENCLAVE_PORT,
    dev_mode=True,
    reset=True,
)

In [ ]:
do_canada_client = canada_node.login(email="info@openmined.org", password="changethis")
do_italy_client = italy_node.login(email="info@openmined.org", password="changethis")

assert do_canada_client.metadata.node_type == NodeType.DOMAIN
assert do_italy_client.metadata.node_type == NodeType.DOMAIN

# Upload datasets to both domains

In [ ]:
# Using public datasets from Freely Extensible Biomedical Record Linkage (Febrl) project
canada_census_data, italy_census_data = load_febrl4()

In [ ]:
for dataset, client, country in zip(
    [canada_census_data, italy_census_data],
    [do_canada_client, do_italy_client],
    ["Canada", "Italy"],
):
    private_data, mock_data = dataset[:2500], dataset[2500:]
    dataset = sy.Dataset(
        name=f"{country} - FEBrl Census Data",
        description="abc",
        asset_list=[
            sy.Asset(
                name="census_data",
                mock=mock_data,
                data=private_data,
                shape=private_data.shape,
                mock_is_real=True,
            )
        ],
    )
    client.upload_dataset(dataset)

In [ ]:
assert len(do_canada_client.datasets.get_all()) == 1
assert len(do_italy_client.datasets.get_all()) == 1

# Create account for data scientist on both the domains

In [ ]:
for client in [do_canada_client, do_italy_client]:
    res = client.register(
        name="Sheldon",
        email="sheldon@caltech.edu",
        password="changethis",
        password_verify="changethis",
    )
    assert isinstance(res, SyftSuccess)

# Register the enclave with Canada domain

In [ ]:
route = HTTPNodeRoute(host_or_ip="localhost", port=CANADA_ENCLAVE_PORT)
do_canada_client.enclaves.add(route=route)

In [ ]:
assert (len(do_canada_client.enclaves.get_all())) == 1
do_canada_client.enclaves.get_all()

In [ ]:
ds_canada_client = sy.login(
    email="sheldon@caltech.edu", password="changethis", port=CANADA_DOMAIN_PORT
)

In [ ]:
# Data scientist should not be able to add enclave to the domain
with pytest.raises(SyftAttributeError) as exc_info:
    ds_canada_client.enclaves.add(
        name="Dummy Enclave", route=HTTPNodeRoute(host_or_ip="localhost", port=9084)
    )
print(exc_info.value)

In [ ]:
# Ensure that the data scientist can access the enclave added by the domain owner
assert (len(ds_canada_client.enclaves.get_all())) == 1
ds_canada_client.enclaves.get_all()

# Cleanup local domain servers

In [ ]:
if canada_node.deployment_type.value == "python":
    canada_node.land()

if italy_node.deployment_type.value == "python":
    italy_node.land()

if canada_enclave.deployment_type.value == "python":
    canada_enclave.land()